<a href="https://colab.research.google.com/github/fritzmartin003/RAG-System-Projekt/blob/main/Hugging_Face_Sara_FineTuning_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Notwendige Bibliotheken installieren
!pip install faiss-cpu transformers sentence-transformers pymupdf numpy scipy


In [10]:
# Imports
import fitz  # PyMuPDF
import numpy as np
import faiss
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer

In [11]:
# PDF-Text extrahieren
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text("text") + "\n"
    return text

pdf_path = "SakowskiBuch.pdf"
pdf_text = extract_text_from_pdf(pdf_path)


In [12]:
# Text in Chunks teilen
def split_text(text, chunk_size=300, overlap=90):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

chunks = split_text(pdf_text)
print(f" PDF in {len(chunks)} Chunks unterteilt!")


 PDF in 2969 Chunks unterteilt!


In [13]:
from google.colab import userdata
HuggingFaceAPIKey = 'HF2'
# Embeddings mit Hugging Face Sentence Transformer
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def get_embedding(text):
    return embedding_model.encode(text, convert_to_numpy=True)

chunk_embeddings = np.array([get_embedding(chunk) for chunk in chunks])


In [ ]:
# FAISS Vektordatenbank aufsetzen
dimension = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(chunk_embeddings)
print(" FAISS Vektordatenbank erstellt!")


✅ FAISS Vektordatenbank erstellt!


In [14]:
# Ähnlichkeitssuche in FAISS
def get_relevant_chunks(query, top_k=3):
    query_embedding = get_embedding(query).reshape(1, -1)
    distances, indices = index.search(query_embedding, top_k)
    return [chunks[i] for i in indices[0]]


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import os
import torch

!huggingface-cli login

os.environ["HUGGINGFACE_API_KEY"] = "HF2"
model_name = "mistralai/Mistral-7B-v0.1"  # Alternativ: "meta-llama/Llama-2-7b-chat-hf"

# Tokenizer & Modell mit explizitem Token laden
pipe = pipeline("text-generation", model="mistralai/Mistral-7B-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")


def generate_answer(query):
    relevant_chunks = get_relevant_chunks(query, top_k=3)
    context = "\n".join(relevant_chunks)
    prompt = f"Beantworte die Frage basierend auf diesem Kontext:\n\n{context}\n\nFrage: {query}\nAntwort:"

    response = pipe(prompt, max_new_tokens=128, temperature=0.7)
    return response[0]["generated_text"]



In [18]:
# Test
frage = "Welche Schutzrechte gibt es für werdende Mütter im Arbeitsrecht?"
antwort = generate_answer(frage)
print("Antwort:", antwort)

NameError: name 'index' is not defined